In [1]:
import numpy as np
import pandas as pd
from sklearn import svm, tree

path_train = '/home/fraga/Documents/Titanic/titanic/train.csv'
path_test = '/home/fraga/Documents/Titanic/titanic/test.csv'

In [2]:
train_df = pd.read_csv(path_train, index_col = 'PassengerId')
train_df.head(10)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S


In [3]:
train_df.isna().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [4]:
def process_train(df):
    y_train = df["Survived"]
    del df["Name"]
    del df["Ticket"]
    del df["Cabin"]
    del df["Survived"]
    
    sex = np.asarray(df["Sex"])
    sex_encode = np.zeros(len(sex), dtype = np.uint8) 

    for i in range(len(sex)):
        if(sex[i] == 'male'):
            sex_encode[i] = 0

        else:
            sex_encode[i] = 1
    
    df["Sex"] = sex_encode
    df["Embarked"].fillna(method = 'ffill', inplace = True)
    age_mean = np.nanmean(np.asarray(df["Age"]))
    df["Age"].fillna(value = age_mean, inplace = True)
    
    embarked = np.asarray(df["Embarked"])
    embarked_encode = np.zeros(len(embarked), dtype = np.uint8)

    for i in range(len(embarked)):
        if(embarked[i] == 'C'):
            embarked_encode = 1

        elif(embarked[i] == 'Q'):
            embarked_encode = 2

        else:
            embarked_encode = 3

    df["Embarked"] = embarked_encode
    
    return y_train

In [5]:
y_train = process_train(train_df)
train_df.head(10)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
1,3,0,22.000000,1,0,7.2500,2
2,1,1,38.000000,1,0,71.2833,2
3,3,1,26.000000,0,0,7.9250,2
4,1,1,35.000000,1,0,53.1000,2
5,3,0,35.000000,0,0,8.0500,2
6,3,0,29.699118,0,0,8.4583,2
7,1,0,54.000000,0,0,51.8625,2
8,3,0,2.000000,3,1,21.0750,2
9,3,1,27.000000,0,2,11.1333,2


In [6]:
train_df.isna().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [7]:
test_df = pd.read_csv(path_test, index_col = 'PassengerId')
test_df.head(10)

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C


In [8]:
test_df.isna().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

In [9]:
def process_test(df):
    del df["Name"]
    del df["Ticket"]
    del df["Cabin"]
    
    sex = np.asarray(df["Sex"])
    sex_encode = np.zeros(len(sex), dtype = np.uint8) 

    for i in range(len(sex)):
        if(sex[i] == 'male'):
            sex_encode[i] = 0

        else:
            sex_encode[i] = 1
    
    df["Sex"] = sex_encode
    
    age_mean = np.nanmean(np.asarray(df["Age"]))
    df["Age"].fillna(value = age_mean, inplace = True)
    
    fare_mean = np.nanmean(np.asarray(df["Fare"]))
    df["Fare"].fillna(value = fare_mean, inplace = True)
    
    embarked = np.asarray(df["Embarked"])
    embarked_encode = np.zeros(len(embarked), dtype = np.uint8)

    for i in range(len(embarked)):
        if(embarked[i] == 'C'):
            embarked_encode = 1

        elif(embarked[i] == 'Q'):
            embarked_encode = 2

        else:
            embarked_encode = 3

    df["Embarked"] = embarked_encode

In [10]:
process_test(test_df)
test_df.isna().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [41]:
def make_submission(train_df, y_train, test_df, arch):
    if(arch == 'svm'):
        model = svm.SVR()
    elif(arch == 'rf'):
        model = tree.DecisionTreeRegressor()
    
    model.fit(train_df, y_train)
    
    predictions = model.predict(test_df)
    submision = []

    for predict in predictions:
        if(predict > 0.5):
            submision.append(1)

        else:
            submision.append(0)

    ind = [i for i in range(892, 1310)]
    
    data = {'PassengerId': ind,
        'Survived': submision
    }

    result = pd.DataFrame(data, columns = ['PassengerId', 'Survived'])
    result.to_csv('number 2.csv', index = False)

In [42]:
make_submission(train_df, y_train, test_df, 'rf')